# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [127]:
# imports
import ollama
import os
from openai import OpenAI
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

In [128]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [129]:
# set up environment
load_dotenv(override=True)
api_key_OpenAI = os.getenv('OPENAI_API_KEY')

In [130]:
# here is the question; type over this to ask something new

system_prompt = """
You are a personal tutor in LLM engineering, AI and Machine Learning. 
Respond the questions not only to give a dirrect solution, 
but explain the concept about which the question is asked, 
or give the framework for solution instead of answering only 
in order to help your student master their AI engineering skills.
Priovide your response in markdown.
"""

question = """
Name 3 main types of quantization for LLM models 
and explain how does they differ?
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]

In [131]:
openai = OpenAI(api_key=api_key_OpenAI)

In [132]:
import tempfile
import subprocess
from io import BytesIO
from pydub import AudioSegment
import time

def play_audio(audio_segment):
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "temp_audio.wav")
    try:
        audio_segment.export(temp_path, format="wav")
        time.sleep(3) # Student Dominic found that this was needed. You could also try commenting out to see if not needed on your PC
        subprocess.call([
            "ffplay",
            "-nodisp",
            "-autoexit",
            "-hide_banner",
            temp_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    finally:
        try:
            os.remove(temp_path)
        except Exception:
            pass
 
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=message
    )
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play_audio(audio)

In [133]:
import base64
from io import BytesIO
from PIL import Image

def artist(flashcard_title):
    image_response = openai.images.generate(
        model="dall-e-2",
        prompt=f'An image associated with the title for a flashcard "{flashcard_title}", in a vibrant pop-art style',
        size="1024x1024",
        n=1,
        response_format="b64_json"
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [134]:
import json
import os

def create_flashcard(JSON_content):
    # Parse JSON and extract front and back
    data = json.loads(JSON_content)
    front = data.get('front', '')
    back = data.get('back', '')

    # Ensure flashcards folder exists in the root directory
    proj_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Go up one directory
    flashcards_folder = os.path.join(proj_root_path, 'flashcards')
    images_folder = os.path.join(flashcards_folder, 'flashcard_images')
    os.makedirs(flashcards_folder, exist_ok=True)
    os.makedirs(images_folder, exist_ok=True)

    # Create a safe filename for the flashcard and image
    safe_name = front[:30].replace(' ', '_').replace('/', '_')
    md_filename = safe_name + ".md"
    image_filename = safe_name + ".png"  # save image as PNG
    md_filepath = os.path.join(flashcards_folder, md_filename)
    image_filepath = os.path.join(images_folder, image_filename)

    # Generate and save the image
    image = artist(front)

    # Save the image
    image.save(image_filepath)

    # Relative path from md file to image (assuming md is in flashcards/, image in flashcard_images/)
    relative_image_path = os.path.join('flashcard_images', image_filename)

    # Write flashcard content with image placeholder
    with open(md_filepath, 'w', encoding='utf-8') as f:
        f.write(front + f"\n\n![Image]({relative_image_path})\n\n" + back)

    print(f"Flashcard saved to {md_filepath}")
    
    return json.dumps({
        "status": "success",
        "flashcard_path": md_filepath,
        "image_path": image_filepath
    })
    
# create_flashcard('{"front": "[TITLE]", "back": "[EXPLANATION]"}')

In [135]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)

    # Here the tool is create_flashcard, expects JSON content as string
    json_content = json.dumps(arguments)  # The arguments dict serialized to string

    # Call your tool function
    tool_response_str = create_flashcard(json_content)  # This returns a JSON string

    response = {
        "role": "tool",
        "content": tool_response_str,
        "tool_call_id": tool_call.id
    }
    return response

In [136]:
flashcard_creator = {
            "name": "create_flashcard",
            "description": "Creates a flashcard with a title and explanation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "front": {"type": "string", "description": "The flashcard title or question"},
                    "back": {"type": "string", "description": "The flashcard answer or explanation"}
                },
                "required": ["front", "back"]
            }
        }

In [137]:
tools = [{"type": "function", "function": flashcard_creator}]

In [ ]:
def chat(history):
    # history is already a list of dicts with "role" and "content"
    messages = [{"role": "system", "content": system_prompt}] + history
    
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        tools=tools
    )
    image = None
    
    # Handle tool calls (e.g., flashcard creation)
    if response.choices[0].finish_reason == "tool_calls":
        tool_message = response.choices[0].message
        tool_response = handle_tool_call(tool_message)
        
        # Append tool call and its response
        messages.append(tool_message)
        messages.append(tool_response)
        
        # Continue chat after tool execution
        response = openai.chat.completions.create(
            model=MODEL_GPT,
            messages=messages
        )
        
        # Parse tool response for image (if flashcard created)
        content_dict = json.loads(tool_response["content"])
        if "flashcard_path" in content_dict:
            image = content_dict["image
            _path"]

    # Extract assistant reply
    reply = response.choices[0].message.content
    history += [{"role": "assistant", "content": reply}]

    # Optionally speak the reply (if talker is defined)
    talker(reply)

    return history, image

In [139]:
# # More involved Gradio code as we're not using the preset Chat interface!
# # Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

import gradio as gr

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(allowed_paths=["/Users/severynkurach/Desktop/Programming/llm_engineering/flashcards"])

* Running on local URL:  http://127.0.0.1:7893
* To create a public link, set `share=True` in `launch()`.


Flashcard saved to /Users/severynkurach/Desktop/Programming/llm_engineering/flashcards/What_are_Decision_Trees_in_Mac.md
